In [1]:
import torch
from d2l import torch as d2l

In [2]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
                  [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [4]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)
    

In [5]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [8]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

通道使我们能够兼具两者的优势：允许显著非线性的多层感知机（MLP）和允许进行局部特征分析的卷积。特别地，通道使 CNN 能够同时处理多个特征，如边缘和形状检测器。它们还在平移不变性和局部性导致的显著参数减少，与计算机视觉中富有表现力且多样化模型的需求之间提供了实际的权衡。

处理多个通道的优势如下：

1. **多样性**：多通道允许 CNN 学习和表示多种特征或属性。这意味着可以根据不同的输入特性（例如颜色、纹理、形状等）进行训练。

2. **模型能力**：多个通道使我们能够学习特征之间的潜在关系，从而通过融合来自不同通道（或特征）的信息来增强网络在预测、分类等任务中的性能。

3. **可组合性**：多通道有助于增加网络的非线性和模型的富有表现力。通过组合来自不同通道的特征，CNN 可以识别和捕捉特征之间的更复杂关系。

4. **参数减少**：多通道提供了一种在保留表达能力的同时减少参数数量的方法，使得模型在统计上和计算上更容易处理。通过在不同通道上重复使用卷积核，可以显著减少所需参数数量。

综上所述，多通道使我们能够为计算机视觉任务建立更强大、更高效且具有非线性特性的模型。